### Install the required packages

In [ ]:
#%pip install python-dotenv
#%pip install requests
#%pip install azure-storage-blob
#%pip install azure-storage-file-datalake

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values,load_dotenv
import sys
sys.path.append('../common')
from helper import Helper

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
region_override = secrets.get("REGION_OVERRIDE")

helper = Helper(environment, "data-transfer/azure/azcopy", api_key, region_override)

### Health Check Tests

In [ ]:
response = helper.get("/health-check/self")

assert response.status_code == 200
assert response.json['status'] == "Healthy"

### Version Tests

In [ ]:
response = helper.get("/version")

assert response.status_code == 200
assert response.json['version'].startswith("1.0")

### Data Transfer Tests - Positive Test Cases

#### Positive End to End Test Case

##### Initialize the test case

In [ ]:
from azure.storage.blob import BlobServiceClient, generate_container_sas, ContainerSasPermissions
from azure.storage.filedatalake import DataLakeServiceClient
from datetime import datetime, timedelta, timezone

source_container_name = 'reg-source'
destination_container_name = 'reg-destination'
directory_name = 'sourceData'

blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)
data_lake_client = DataLakeServiceClient.from_connection_string(account_connection_string)

account_name = blob_service_client.account_name

##### Delete any previous destination contents

In [ ]:
if (data_lake_client.get_file_system_client(destination_container_name).get_directory_client(directory_name).exists()):
    data_lake_client.get_file_system_client(destination_container_name).delete_directory(directory_name)

##### Generate source and destination SAS URIs

In [ ]:
source_sas_token = generate_container_sas(
    account_name=account_name,
    container_name=source_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=1)
)

destination_sas_token = generate_container_sas(
    account_name=account_name,
    container_name=destination_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True, write=True, delete=True, add=True, create=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=1)
)

source_sas_url = f"https://{account_name}.blob.core.windows.net/{source_container_name}?{source_sas_token}"
destination_sas_url = f"https://{account_name}.blob.core.windows.net/{destination_container_name}?{destination_sas_token}"

helper.log(f"Source SAS URL: {source_sas_url}")
helper.log(f"Destination SAS URL: {destination_sas_url}")

##### Start the Data Transfer

In [ ]:
response = helper.post("/azure/azcopysync", {
    "sourceSasUri": source_sas_url,
    "destinationSasUri": destination_sas_url,
    "recursive":True
})

assert response.status_code == 202

ingestionId = response.json['ingestionJobId']

##### Poll for the status of the job

In [ ]:
import time

done = False

while(done != True):
    response = helper.get(f'/azure/azcopysync/{ingestionId}')
    done = response.json['jobStatus'] == "Completed" or response.json['jobStatus'] == "Failed"
    time.sleep(3)

assert response.status_code == 200
assert response.json['ingestionJobId'] == ingestionId
assert response.json['jobStatus'] == "Completed"
assert response.json['detailedStatus']["totalTransfers"] > 0